In [1]:
import pandas as pd
import yfinance as yf
from numpy import nan
import requests
import datetime

In [ ]:
start = datetime.datetime.now()

#import screen criteria
settings = pd.read_csv('dividend_settings.txt',)
mindiv = float(settings.iloc[0])
minchowder = float(settings.iloc[2])
maxpe = float(settings.iloc[4])
maxpayout = float(settings.iloc[6])
maxdebt = float(settings.iloc[8])

In [ ]:
print('Downloading Dividend Champions Sheet...\n')

### READ DIRECTLY WITH PANDAS ###
#obtain latest copy of USDividendChampions
url = 'https://bitly.com/USDividendChampions'
file = requests.get(url)

with open('DividendChampions.xlsx','wb') as d:
    d.write(file.content)

In [ ]:
print('Processing Dividend Champions Spreadsheet...\n')

#read in the dividend stock data
filename = 'DividendChampions.xlsx'
df1 = pd.read_excel(filename,sheet_name='Champions')
df2 = pd.read_excel(filename,sheet_name='Contenders')
df3 = pd.read_excel(filename,sheet_name='Challengers')
df_list = [df1,df2,df3]

In [ ]:
#clean data
for df in df_list:
    df.drop(df.index[0:5], inplace=True)
    df.columns = ['Company','Ticker','Sector','Industry','No.Yrs','CCCSeq','DRIP DR', 'Fees SP','Price', 'Div.Yield', 'Current Dividend','Payouts/Year', 'Annualized','Qtly Sch','Previous Payout','Last Increased on:Ex-Div','Last Increased on:Pay','MR%Inc.','DGR 1-yr','DGR 3-yr','DGR 5-yr','DGR 10-yr','5/10 A/D*','Past 5yr DEG','&=MultiIncThisYrNotes','EPS %Payout','TTM P/E','FYE Month','TTM EPS','PEG','TTM P/Sales','MRQ P/Book','TTM ROE','TTM Growth','NY Growth','Past 5yr Growth','Est-5yr Growth','MktCap($Mil)','Inside Own.','Debt/Equity','TweedFactor','Chowder Rule','+/-% vs.Graham','Estimated Div:2018','2019','2020','2021','2022','Est. Payback$','Est. Payback%','5-yr Beta','52-wk Low','52-wk High','50-day MMA','200-day MMA','OTC','StreakBegan','RecessionsSurvived','TTM ROA']
    df.drop(df[df['Company']==nan].index,inplace=True)
    df.drop(df.index[-15:], inplace=True)
#concatenate all df's into master df
df = pd.concat(df_list)
print('Total # of Stocks considered:',df.shape[0],'\n')
      
#convert datatype for columns of interest
scr_col = ['Div.Yield','EPS %Payout','Debt/Equity','TTM P/E','Chowder Rule','DGR 1-yr','DGR 3-yr','DGR 5-yr','DGR 10-yr','5/10 A/D*']
for col in scr_col:
    df[col] = df[col].astype(float)

In [ ]:
print('Fetching latest stock prices...\n')

#get latest stock price
ticker_list = list(df.Ticker.unique())
ticker_list_clean = [ticker for ticker in ticker_list if isinstance(ticker, str)]
quote_date = start.strftime('%Y-%m-%d')
start_date = start - datetime.timedelta(days=1)
print(quote_date, start_date)
dat = yf.download(ticker_list_clean,start=start_date,end=quote_date,group_by='ticker')
print(dat.info())